In [ ]:
DATABASE_PATH = '../database/abundances.db'
element_list = ['Fe', 'Ti', 'Ca', 'Si', 'Al', 'Mg', 'Na'] + \
    ['Plagioclase Feldspar', 'Olivine', 'Ilmenite', 'Armalcolite']

In [2]:
FETCH_LIMIT = None

In [3]:
import json
import sqlite3



import sqlite3
import pandas as pd
from flask import jsonify

def query_db(query, params=(), limit=None):
    db_path = DATABASE_PATH
    connection = sqlite3.connect(db_path)

    # Add LIMIT clause if a limit is specified
    if limit is not None:
        query += f" LIMIT {limit}"
        
    # Using pandas to read the query result directly into a DataFrame
    df = pd.read_sql_query(query, connection, params=params)

    connection.close()

    # Convert DataFrame to a list of dictionaries
    result_list = df.to_dict(orient='records')

    return result_list


In [4]:
query = "SELECT * FROM abundances"
data = []

In [5]:
import time

In [6]:
# Start the timer to measure the query execution time

start_time = time.time()

result = query_db(query, data,FETCH_LIMIT )


end_time = time.time()
# Print the time taken to perform the query
print(f"time to fetch : {end_time - start_time:.4f} seconds")

time to fetch : 3.0134 seconds


In [7]:
len(result) / 7

62363.0

In [8]:
import sys


# Get the size of the variable
size_in_bytes = sys.getsizeof(result)

print(f"Size of my_list: {size_in_bytes//1024**2} MB")


Size of my_list: 3 MB


In [9]:
import time
from sklearn.neighbors import KDTree
import numpy as np

points = result

In [10]:
# kernel = True

In [11]:
# Create a KDTree for each element
element_trees = {}
element_points_dict = {}

# Start the timer to measure the query execution time
start_time = time.time()

for element in element_list:
    # print("creating for element", element)
    element_points = [
        point for point in points if point['element'] == element]
    element_points_dict[element] = element_points
    element_points = [(point['lat'], point['long'])
                        for point in element_points]
    element_trees[element] = KDTree(np.array(element_points))

end_time = time.time()
# Print the time taken to perform the query
print(f"Trees creation time : {end_time - start_time:.4f} seconds")

# Query function
def get_nearby_points(query_point, radius_degrees, element):
    query_array = np.array([query_point])

    # Query the relevant tree for the specified element
    tree = element_trees.get(element)
    if tree:
        indices = tree.query_radius(query_array, r=radius_degrees)
        return [points[i] for i in indices[0]]
    else:
        return []



Trees creation time : 0.4752 seconds


In [12]:
len(element_points_dict['Mg'])

62363

## inspecting dates

In [13]:
s1 = {point['date'] for point in points}
# Convert to a list
s1 = sorted(list(s1))
s1

# Save to JSON
with open("../static/dates.json", "w") as f:
    json.dump(s1, f, indent=4)

print("Unique dates saved to 'dates.json'.")

Unique dates saved to 'dates.json'.


## histograms

In [14]:
# Initialize a dictionary to store histogram data
histograms = {}

# Define bins
bins = np.arange(0, 51, 5)  # 0-10, 10-20, ..., 90-100

# Compute histograms with normalization
for element, points in element_points_dict.items():
    # Extract abundances
    abundances = [entry["abundance"] for entry in points]
    
    # Compute histogram counts
    counts, _ = np.histogram(abundances, bins=bins)
    
    # Normalize counts
    total = sum(counts) 
    normalized_counts = (counts*100 / total).tolist() if total > 0 else counts.tolist()
    
    # Store the result in the histogram dictionary
    histograms[element] = normalized_counts

histograms


histograms

{'Fe': [72.980773856293,
  21.907220627615732,
  4.239693407950227,
  0.8723121081410452,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'Ti': [50.9661177300643,
  31.06649776309671,
  15.398553629555987,
  2.568830877283004,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'Ca': [0.0,
  89.25805365360871,
  9.563362891458077,
  1.1785834549332137,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'Si': [30.27917194490323,
  30.74739829706717,
  23.19644661097125,
  11.450699934255889,
  3.5742347225117457,
  0.5419880377788111,
  0.1603514904671039,
  0.041691387521447014,
  0.008017574523355194,
  0.0],
 'Al': [0.0,
  9.463944967368471,
  40.48233728332505,
  50.052114234401806,
  0.001603514904671039,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'Mg': [7.815531645366644,
  20.15778586661963,
  34.3055978705322,
  37.721084617481516,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'Na': [100.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}

In [15]:
# Save to JSON
with open("../static/histograms.json", "w") as f:
    json.dump(histograms, f, indent=4)

print("Histograms computed and saved to 'histograms.json'.")

Histograms computed and saved to 'histograms.json'.


In [16]:
from sklearn.cluster import KMeans
import numpy as np



In [17]:
len(element_points)

62363

In [18]:
# raise End

## Clustering

In [ ]:
# takes around 5 minutes to run

In [19]:
# Perform clustering to create 2k spatial clusters
coordinates = element_points
print(len(coordinates))
n_clusters = 5000
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
labels = kmeans.fit_predict(coordinates)

# Save cluster labels and centers
cluster_centers = kmeans.cluster_centers_



62363


c:\Users\pande\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [20]:
import numpy as np
from collections import defaultdict

# Initialize variables
M = len(element_points_dict)  # Number of elements
n_clusters = max(labels)+1  # Total distinct labels

# Create a structure to store the cluster means for each element
cluster_means_per_element = {element: [0] * n_clusters for element in element_points_dict}

# Iterate over each element and compute cluster means
for element, points in element_points_dict.items():
    # Create a list to store sums and counts for each cluster
    cluster_sums = [0] * n_clusters
    cluster_counts = [0] * n_clusters

    # Accumulate abundances for each cluster
    for point, label in zip(points, labels):
        cluster_sums[label] += point['abundance']
        cluster_counts[label] += 1

    # Compute means for each cluster
    cluster_means_per_element[element] = [
        cluster_sums[i] / cluster_counts[i] if cluster_counts[i] > 0 else 0 for i in range(n_clusters)
    ]

# Debug: Print a small sample of results
for element, means in cluster_means_per_element.items():
    print(f"Cluster means for {element}: {means[:10]}")  # Print the first 10 clusters as a sample


Cluster means for Fe: [3.4580784134339253, 1.6191440099067331, 6.0480330175917665, 7.186616735478911, 8.205664633921016, 1.9084488004680635, 4.179363450760282, 1.7554068871041397, 3.8485371261271752, 3.4714702206938823]
Cluster means for Ti: [2.486855749277567, 2.221998212913582, 5.894939800616006, 2.9039165454687814, 1.9874565396086843, 4.740308720874544, 3.1170478446779066, 1.7207338496780609, 4.458843059095832, 2.012771507839379]
Cluster means for Ca: [5.66647998495298, 5.4284696876457135, 11.653245384641517, 6.508342600621223, 13.25252738141349, 6.676677462924633, 10.309933666358857, 5.408293989450935, 5.7916176153536805, 7.05037061506221]
Cluster means for Si: [12.063072544043587, 8.783423616438638, 2.0707916871350034, 8.881607019005365, 8.452254078278806, 12.252874037038104, 14.566960946584384, 10.950878818698719, 10.618328232554875, 12.202647131517772]
Cluster means for Al: [14.033638438223402, 17.0787317383895, 13.52904994748344, 17.55213796240211, 9.468150854197608, 15.0487580

In [21]:
cluster_means_per_element['Fe']

[3.4580784134339253,
 1.6191440099067331,
 6.0480330175917665,
 7.186616735478911,
 8.205664633921016,
 1.9084488004680635,
 4.179363450760282,
 1.7554068871041397,
 3.8485371261271752,
 3.4714702206938823,
 10.481499321947966,
 4.0294208215393255,
 5.92565909370979,
 2.3220529073282647,
 2.0202252098330242,
 2.690763309245029,
 1.815050650559904,
 6.79189203718565,
 1.9498342964059567,
 4.2435932867406745,
 2.186599870596982,
 3.8295189145277684,
 1.3116689053044839,
 3.5705303418463097,
 1.3476864021848411,
 1.9469429206522089,
 4.650315087050318,
 1.3293936415703005,
 3.994027280094772,
 1.8158350997629937,
 1.9222810624076543,
 3.954580939373927,
 1.4004937455411444,
 2.1683564194804146,
 6.704456448874998,
 2.85665207162635,
 1.7050780066430742,
 5.05704928372821,
 2.871666579374138,
 3.022235171593458,
 2.0894498660210528,
 8.350542466230108,
 9.541585445729055,
 8.24679738214103,
 1.602495497332207,
 6.016840029134525,
 4.940858479514194,
 6.882319094082687,
 2.292028008163805,


In [22]:
cluster_centers

array([[ -59.14088833,   56.043185  ],
       [  18.8150875 , -107.48375   ],
       [  49.07005625,   43.53769844],
       ...,
       [  73.17109722, -125.266     ],
       [  43.1479375 ,  117.802125  ],
       [ -61.45330781,  -98.90729688]])

In [23]:
# import json

# # Save the elementwise_cluster_data to a JSON file
# with open('static/{element}_clusters.json', 'w') as json_file:
#     json.dump(elementwise_cluster_data, json_file, indent=4)

# print("Data saved to 'elementwise_cluster_data.json'")


In [24]:
# Initialize a dictionary to store the final result
elementwise_cluster_data = {}

# Iterate through each element and its corresponding cluster means
for element, cluster_means in cluster_means_per_element.items():
    elementwise_cluster_data[element] = [
        {
            'element': element,
            'abundance': cluster_means[i],
            'lat': cluster_centers[i][0],  # Latitude of the cluster center
            'long': cluster_centers[i][1]  # Longitude of the cluster center
        }
        for i in range(len(cluster_centers))
    ]

# Example: Print the first cluster data for a specific element (e.g., 'Fe')
print(f"First cluster for Fe: {elementwise_cluster_data['Fe'][0]}")


First cluster for Fe: {'element': 'Fe', 'abundance': 3.4580784134339253, 'lat': -59.14088833333333, 'long': 56.043185}


In [25]:
import json

# Save the elementwise_cluster_data to a JSON file
with open('../static/clusters.json', 'w') as json_file:
    json.dump(elementwise_cluster_data, json_file, indent=4)

print("Data saved to 'clusters.json'")


Data saved to 'clusters.json'


In [26]:
len(cluster_centers)

5000

# End of file